In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
load_dotenv()
from pprint import pprint

In [2]:
groq_api_key = os.environ.get('GROQ_API_KEY')


In [3]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [4]:
from langchain.chains import create_sql_query_chain

llm = ChatGroq(model="llama-3.3-70b-versatile")
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many Genre are there and who is the best artist in each genre?, do not use limit function in query"})
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parsed = parser.parse(response)

In [5]:
print(parsed)

To answer this question, we need to find out the number of genres and the best artist in each genre. 

First, let's find out the number of genres. We can do this by querying the "Genre" table.

```sql
SELECT COUNT("GenreId") 
FROM "Genre"
```

To find out the best artist in each genre, we need to join the "Genre", "Track", and "Album" tables.

```sql
SELECT "G"."Name" AS "GenreName", "A"."Name" AS "BestArtist" 
FROM "Genre" "G" 
JOIN "Track" "T" ON "G"."GenreId" = "T"."GenreId" 
JOIN "Album" "AL" ON "T"."AlbumId" = "AL"."AlbumId" 
JOIN "Artist" "A" ON "AL"."ArtistId" = "A"."ArtistId"
```

However, this query will return all artists in each genre, not the best artist. To find the best artist, we need to define what "best" means. Let's assume the best artist is the one with the most tracks in a genre.

```sql
SELECT "G"."Name" AS "GenreName", "A"."Name" AS "BestArtist" 
FROM "Genre" "G" 
JOIN "Track" "T" ON "G"."GenreId" = "T"."GenreId" 
JOIN "Album" "AL" ON "T"."AlbumId" = "AL"."AlbumId

In [6]:
import re

def parse_query(parsed):
    # Response text
    response_text = parsed
    
    # Regex to find SQL code blocks
    sql_blocks = re.findall(r"```sql\n(.*?)\n```", response_text, re.DOTALL)
    
    # Print or save the extracted SQL queries
    for i, sql_query in enumerate(sql_blocks, start=1):
        print(f"Query {i}:\n{sql_query}\n")
    
    return sql_blocks

sql_blocks = parse_query(parsed)

Query 1:
SELECT COUNT("GenreId") 
FROM "Genre"

Query 2:
SELECT "G"."Name" AS "GenreName", "A"."Name" AS "BestArtist" 
FROM "Genre" "G" 
JOIN "Track" "T" ON "G"."GenreId" = "T"."GenreId" 
JOIN "Album" "AL" ON "T"."AlbumId" = "AL"."AlbumId" 
JOIN "Artist" "A" ON "AL"."ArtistId" = "A"."ArtistId"

Query 3:
SELECT "G"."Name" AS "GenreName", "A"."Name" AS "BestArtist" 
FROM "Genre" "G" 
JOIN "Track" "T" ON "G"."GenreId" = "T"."GenreId" 
JOIN "Album" "AL" ON "T"."AlbumId" = "AL"."AlbumId" 
JOIN "Artist" "A" ON "AL"."ArtistId" = "A"."ArtistId"
GROUP BY "G"."Name", "A"."Name"
ORDER BY COUNT("T"."TrackId") DESC

Query 4:
SELECT "GenreName", "BestArtist" 
FROM (
    SELECT "G"."Name" AS "GenreName", "A"."Name" AS "BestArtist", 
    COUNT("T"."TrackId") AS "TrackCount",
    ROW_NUMBER() OVER (PARTITION BY "G"."Name" ORDER BY COUNT("T"."TrackId") DESC) AS "RowNum"
    FROM "Genre" "G" 
    JOIN "Track" "T" ON "G"."GenreId" = "T"."GenreId" 
    JOIN "Album" "AL" ON "T"."AlbumId" = "AL"."AlbumId" 
 

In [7]:
len(sql_blocks)

6

In [8]:
# sql_blocks = []
if sql_blocks:
    # sql_blocks[-1]
    try:
        print(db.run(sql_blocks[-1]))
    except Exception as e:
        print(e)
else: print("---")

[('Number of Genres', 25), ('Alternative', 'Chris Cornell'), ('Alternative & Punk', 'Titãs'), ('Blues', 'Eric Clapton'), ('Bossa Nova', 'Toquinho & Vinícius'), ('Classical', 'Eugene Ormandy'), ('Comedy', 'The Office'), ('Drama', 'Lost'), ('Easy Listening', 'Frank Sinatra'), ('Electronica/Dance', 'O Rappa'), ('Heavy Metal', 'Iron Maiden'), ('Hip Hop/Rap', 'House Of Pain'), ('Jazz', 'Miles Davis'), ('Latin', 'Os Paralamas Do Sucesso'), ('Metal', 'Metallica'), ('Opera', 'Sir Georg Solti, Sumi Jo & Wiener Philharmoniker'), ('Pop', 'U2'), ('R&B/Soul', 'James Brown'), ('Reggae', 'Cidade Negra'), ('Rock', 'Led Zeppelin'), ('Rock And Roll', 'BackBeat'), ('Sci Fi & Fantasy', 'Battlestar Galactica (Classic)'), ('Science Fiction', 'Battlestar Galactica'), ('Soundtrack', 'Various Artists'), ('TV Shows', 'Lost'), ('World', 'João Suplicy')]


In [9]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [10]:
from langchain_community.tools import QuerySQLDatabaseTool

execute_query = QuerySQLDatabaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query
parsed1 = parser.parse(chain.invoke({"question": "How many employees are there"})) #.split(':')[-1])
# sql_blocks1 = parse_query(parsed1)
# sql_blocks1
parsed1

'Question: How many employees are there\nSQLQuery: SELECT COUNT("EmployeeId") FROM "Employee"'

In [11]:
db.run(' SELECT COUNT("EmployeeId") FROM "Employee" LIMIT 1;')
execute_query.invoke(chain.invoke({"question": "How many employees are there"})
                     .split(':')[-1]
                     )

'[(8,)]'

In [12]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

        Question: {question}
        SQL Query: {query}
        SQL Result: {result}
        Answer: """
    )

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough
    .assign(query=write_query)
    .assign(result=itemgetter("query")
        | execute_query 
    )
    # | answer 
)


chain.invoke({"question": "How many employees are there"})

{'question': 'How many employees are there',
 'query': 'Question: How many employees are there\nSQLQuery: SELECT COUNT("EmployeeId") FROM "Employee"',
 'result': 'Error: (sqlite3.OperationalError) near "Question": syntax error\n[SQL: Question: How many employees are there\nSQLQuery: SELECT COUNT("EmployeeId") FROM "Employee"]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'}

In [13]:
from langchain.chains import create_sql_query_chain
db_chain = create_sql_query_chain(llm, db)
response = db_chain.invoke({"question": "How many employees are there ?"})

In [14]:
response

'Question: How many employees are there ?\nSQLQuery: SELECT COUNT("EmployeeId") FROM "Employee"'

In [15]:
cleaned_query = llm.invoke(f"extract the final query syntax from this: {response} . No preamble , not any extra stuff.")

In [16]:
cleaned_query.content

'SELECT COUNT("EmployeeId") FROM "Employee"'

In [17]:
db.run(cleaned_query.content)

'[(8,)]'

In [18]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [19]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [20]:
toolkit.get_tools()

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f5ba04bd150>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f5ba04bd150>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f5ba04bd150>),
 QuerySQLCheckerTool(description='Use this tool to double check

In [21]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)

/home/dnk131/miniconda3/envs/snode/lib/python3.10/site-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


['dialect', 'top_k']


In [22]:
from pprint import pprint
pprint(prompt_template.pretty_print())

================================ System Message ================================

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to th

In [23]:
import re

def extract_sql_queries(llm_response):
    """
    Extracts all SQL queries from the given LLM response in markdown format.

    Args:
        llm_response (str): The LLM's markdown response containing SQL queries.

    Returns:
        list: A list of extracted SQL queries.
    """
    # Regular expression to match SQL queries
    sql_pattern = r"```sql\s+(.*?)\s+```"
    matches = re.findall(sql_pattern, llm_response, re.DOTALL)

    # Strip and return the SQL queries
    return [match.strip() for match in matches]

# Example LLM response
llm_response ='''
'Here is the answer:\n\nQuestion: How many employees are there\nSQLQuery: SELECT COUNT("EmployeeId") FROM "Employee" LIMIT 1;',
Here are the SQL queries:

```sql
SELECT * FROM users WHERE age > 30;
```

Certainly! Here’s a simple SQL query example that retrieves data from a table called `employees`. This query selects the names and job titles of all employees:

```sql
SELECT name, job_title
FROM employees;
```

### Explanation:
- **SELECT**: This clause specifies the columns you want to retrieve. In this case, `name` and `job_title`.
- **FROM**: This clause specifies the table from which to retrieve the data, which is `employees` in this example.

If you need a specific query or have a particular scenario in mind, feel free to ask!
'''

In [24]:
sql_queries = extract_sql_queries(llm_response)
print("Extracted SQL Queries:")
for q in sql_queries:
    print(q)

Extracted SQL Queries:
SELECT * FROM users WHERE age > 30;
SELECT name, job_title
FROM employees;
